In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import random
from sklearn.neural_network import MLPRegressor

In [2]:
prod_data_train = pd.read_csv('production_data_train.csv')
prod_data_test = pd.read_csv('production_data_test.csv')
ihs_data = pd.read_csv('IHS_data.csv')
harmony_data = pd.read_csv('Harmony_data.csv')
test_apis = pd.read_csv('test_APIs.csv', header=None)
test_apis.columns = ['API']
test_apis['API'] = (test_apis['API']).astype(str)
test_apis['API'] = test_apis['API'].apply(lambda x: x.zfill(14))
sample_file = pd.read_csv('sample_file.csv')    

In [3]:
# preprocessing data
def preprocess(prod_data, wells_data, train):
    prod_data = prod_data.drop_duplicates(subset=['API', 'Month', 'Year'], keep='last', inplace=False)
    
    # adding zeros to API
    prod_data['API'] = (prod_data['API']).astype(str)
    prod_data['API'] = prod_data['API'].apply(lambda x: x.zfill(14))
    
    # adding zeros to API
    wells_data['API'] = (wells_data['API']).astype(str)
    wells_data['API'] = wells_data['API'].apply(lambda x: x.zfill(14))
    
    # storing peak records 
    idx_max = prod_data.groupby(['API'])['Liquid'].transform('max') == prod_data['Liquid']
    max_month_prod_data = prod_data[idx_max].drop_duplicates(subset='API', keep='first', inplace=False)
    list_indices = ['API', 'Year', 'Month', 'Liquid']
    max_month_prod_data = max_month_prod_data[list_indices]
    max_month_prod_data = max_month_prod_data.rename(columns={"Year": "Max_Year", "Month": "Max_Month", "Liquid": "Max_Liquid"})
    
    # merging the two dataframes to get max month and max year
    new_prod_data_orig = prod_data.merge(max_month_prod_data, on='API')

    # Remove Pre-Peak Months (clean up)
    new_prod_data = new_prod_data_orig[((new_prod_data_orig['Year'] == new_prod_data_orig['Max_Year']))]
    new_prod_data = new_prod_data[(new_prod_data['Month'] >= new_prod_data['Max_Month'])]

    new_prod_data2 = new_prod_data_orig[((new_prod_data_orig['Year'] > new_prod_data_orig['Max_Year']))]
    new_prod_data3 = new_prod_data.append(new_prod_data2)
    
    # adding month index column to post peak production data
    new_prod_data3['index'] = calc_month_index(new_prod_data3['Max_Year'], new_prod_data3['Max_Month'], new_prod_data3['Year'], new_prod_data3['Month'])
    indexed_prod_data = new_prod_data3
    
    # removed nullified SpudDates and CompletionDates
    wells_data = wells_data[~((wells_data['SpudDate'].isnull()) & (wells_data['CompletionDate'].isnull()))]
    
    # replacing null CompletionDates with SpudDates + six months
    wells_data['SpudDate'] = pd.to_datetime(wells_data['SpudDate'])
    wells_data['CompletionDate'] = pd.to_datetime(wells_data['CompletionDate'])
    wells_data.loc[wells_data['CompletionDate'].isnull(), 'CompletionDate'] = wells_data['SpudDate'] + timedelta(days=170) 
    
    # replacing StateNames with indices
    unique_state_names = wells_data.StateName.unique()
    unique_state_ids = list(range(0, len(unique_state_names)))
    dict_state_names = dict(zip( unique_state_names, unique_state_ids))
    wells_data['StateName'] = wells_data['StateName'].map(dict_state_names)
                            
    # replacing CountyNames with indices                         
    unique_county_names = wells_data.CountyName.unique()
    unique_county_ids = list(range(0, len(unique_county_names)))
    dict_county_names = dict(zip(unique_county_names, unique_county_ids))
    wells_data['CountyName'] = wells_data['CountyName'].map(dict_county_names)
    
    # replacing BasinName with indices
    unique_basin_names = wells_data.BasinName.unique()
    unique_basin_ids = list(range(0, len(unique_basin_names)))
    dict_basin_names = dict(zip(unique_basin_names, unique_basin_ids))
    wells_data['BasinName'] = wells_data['BasinName'].map(dict_basin_names)
    
    #replacing Formation with indices
    unique_formation_names = wells_data.formation.unique()
    unique_formation_ids = list(range(0, len(unique_formation_names)))
    dict_formation_names = dict(zip(unique_formation_names, unique_formation_ids))
    wells_data['formation'] = wells_data['formation'].map(dict_formation_names)
        

    indexed_prod_data = indexed_prod_data.merge(wells_data, on='API')
    three_years_data = indexed_prod_data
    if(train):
        # removing immature wells
        three_years_data = indexed_prod_data[(indexed_prod_data['Max_Year'] < 2016) |((indexed_prod_data['Max_Year'] == 2016) & (indexed_prod_data['Max_Month'] == 1))] 
        three_years_data = three_years_data[three_years_data['index'] <= 36] 
    
    return three_years_data

In [4]:
# calculates month index
def calc_month_index(max_year, max_month, year, month):
    return (12 - max_month + (year - max_year - 1)*12 + month) * (year != max_year) + (year == max_year) * (month - max_month) + 1 

In [5]:
processed_train = preprocess(prod_data_train, ihs_data, True)
processed_test = preprocess(prod_data_test, ihs_data, False)

harmony_data['API'] = (harmony_data['API']).astype(str)
harmony_data['API'] = harmony_data['API'].apply(lambda x: x.zfill(14))

harmony_data[harmony_data['WATER_PER_FOOT'].isnull()]['WATER_PER_FOOT'] = harmony_data['WATER_PER_FOOT'].mode()
harmony_data[harmony_data['PROP_PER_FOOT'].isnull()]['PROP_PER_FOOT'] = harmony_data['PROP_PER_FOOT'].mode()

C:\Users\Dell\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Dell\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Dell\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [6]:
complete_prod_train = processed_train.merge(harmony_data, on='API')
complete_prod_test = processed_test.merge(harmony_data, on='API', how='outer')
complete_prod_test.fillna(0, inplace=True)
complete_prod_test = complete_prod_test.merge(test_apis, on='API')

# below were attempts to find better replacements for nan values

# complete_prod_test[complete_prod_test['GOR_30'].isnull()]['GOR_30'] = complete_prod_test['GOR_30'].mode()
# complete_prod_test[complete_prod_test['GOR_60'].isnull()]['GOR_60'] = complete_prod_test['GOR_60'].mode()
# complete_prod_test[complete_prod_test['GOR_90'].isnull()]['GOR_90'] = complete_prod_test['GOR_90'].mode()

# complete_prod_train[complete_prod_train['GOR_30'].isnull()]['GOR_30'] = complete_prod_train['GOR_30'].mode()
# complete_prod_train[complete_prod_train['GOR_60'].isnull()]['GOR_60'] = complete_prod_train['GOR_60'].mode()
# complete_prod_train[complete_prod_train['GOR_90'].isnull()]['GOR_90'] = complete_prod_train['GOR_90'].mode()

# replacing gor nans with zeros
complete_prod_train.fillna(0, inplace=True)

In [7]:
cols_train = ['Gas', 'Water', 'Max_Year','operatorNameIHS', 'CompletionDate', 'FirstProductionDate', 'Max_Liquid', 
              'Max_Month', 'SpudDate', 'PermitDate', '_LastUpdate','BasinName', 'StateName', 'CountyName', 'DaysOn', 'WATER_PER_FOOT', 'PROP_PER_FOOT', 'GOR_30', 'GOR_60', 'GOR_90']
cols_test = ['Gas', 'date', 'Water', 'Max_Year','operatorNameIHS', 'CompletionDate', 'FirstProductionDate', 'Max_Liquid',
             'Max_Month', 'SpudDate', 'PermitDate', '_LastUpdate', 'BasinName', 'StateName','CountyName', 'DaysOn', 'WATER_PER_FOOT', 'PROP_PER_FOOT', 'GOR_30', 'GOR_60', 'GOR_90']
complete_prod_train = complete_prod_train.drop(cols_train, axis=1)
complete_prod_test = complete_prod_test.drop(cols_test, axis=1)

In [23]:
## Data Prep below:
## inputing windows of 3 months and y_label is corresponding cum_sum of liquid

testing_apis = []
counter = []
def get_three_months(prod_data, feat_arr, y_label, test):
    prod_data = (prod_data.groupby('API').apply(generate_input, feat_arr, y_label, test))
    
def generate_input(group, feat_arr, y_label, test):
    global testing_apis
    global counter
    if group['API'].max() in testing_apis:
        return 
    count_cur = 0
    testing_apis.append(group['API'].max())
    if len(group) < 2:
        print("API", group['API'])
        
    group = group.drop(['API'], axis=1)
    records = group[:3]
        
    if len(group) >= 3:
        for i in range(len(group) - 2): # 0 1 2
            records = group[i:i+3]
            feat_arr.append(list(np.array(records).flatten()))
            y_label.append(group['Liquid'].sum())
            count_cur += 1
        
    elif test:
        idx_1 = records[records['index'] == 1]
        idx_2 = records[records['index'] == 2]
        idx_3 = records[records['index'] == 3]
        count_cur = 1

        # If month idx 2 missing ==> missing values replaced by average
        missing_rec = idx_1
        missing_rec['Liquid'] =  (idx_1['Liquid'].values[0] + idx_3['Liquid'].values[0])/2
        missing_rec['index'] = 2
        arr = np.vstack((idx_1, missing_rec, idx_3))
        feat_arr.append(list(arr.flatten()))
        y_label.append(group['Liquid'].sum())
        
    counter.append(count_cur)

In [41]:
input_feats = []
y_labels = []
get_three_months(complete_prod_train, input_feats, y_labels, False)
input_feats = np.array(input_feats)
y_labels = np.array(y_labels)
# print("before validation split", input_feats.shape)
(input_feats, X_valid) = input_feats[40000:], input_feats[:40000]
(y_labels, Y_valid) = y_labels[40000:], y_labels[:40000]
print("after validation split", input_feats.shape)

after validation split (226266, 36)


In [62]:
# prepare testing data
x_test = []
testing_apis = []
counter = []
get_three_months(complete_prod_test, x_test, [], True)


In [57]:
# using regessor model
regressor = MLPRegressor(hidden_layer_sizes=(250,50), activation='relu', solver='adam', 
                         alpha=0.0001, batch_size='auto',
                         learning_rate_init=0.001, power_t=0.5, max_iter=200, 
                         shuffle=True, random_state=3, tol=0.0001, verbose=False, 
                         warm_start=False, momentum=0.9, nesterovs_momentum=True, 
                         early_stopping=True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, 
                         epsilon=1e-08)
regressor.fit(input_feats, y_labels)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(250, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=3, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [58]:
predictions = regressor.predict(X_valid)
regressor.score(X_valid, Y_valid) #0.8050442257646431 200 0.7972875790314738 100  250 0.823 350 0.81

0.8237888482131224

In [63]:
predictions = regressor.predict(x_test)

In [72]:
predictions = np.array(predictions)
idx = 0
sum_liquids = {}
for i in range(len(testing_apis)):
    api_cur = testing_apis[i]
    cur_count = counter[i]
    cur_sum = 0
    for j in range(cur_count):
        cur_sum += predictions[idx]
        idx += 1
    sum_liquids[api_cur] = cur_sum/cur_count

In [73]:
data = []
for i in range(len(testing_apis)):
    api_cur = testing_apis[i]
    data.append([testing_apis[i], sum_liquids[api_cur]])
output_df = pd.DataFrame(data, columns = ['Id', 'Predicted'])
output_df.to_csv('three_yrs_cum.csv',index=False)

In [74]:
len(sum_liquids.keys())

3331

In [75]:
sum_liquids

{'05001097740200': 77316.55456860653,
 '05001097770000': 111255.74078300883,
 '05001097790000': 139240.70037981967,
 '05001097810000': 18208.16432782567,
 '05001097850100': 16287.484434183558,
 '05001098010100': 334446.1470595358,
 '05001098260100': 137349.1803004891,
 '05005072030100': 58925.182035328566,
 '05005072180000': 273640.50062009896,
 '05005072200000': 216382.88173762744,
 '05005072230000': 245294.71685407474,
 '05005072240000': 229300.58052094886,
 '05123340180000': 89684.93898374836,
 '05123343840000': 83313.79752524714,
 '05123346950000': 40960.52678035685,
 '05123349520000': 126787.01428956032,
 '05123352520000': 75093.09453646663,
 '05123352980000': 80292.16887404755,
 '05123353520000': 54662.006042829096,
 '05123355430000': 51621.370381155706,
 '05123358360000': 68545.38437568866,
 '05123359980000': 60796.62187759581,
 '05123360020000': 158628.08230043977,
 '05123360790000': 66337.3583184662,
 '05123360830000': 62700.61006322723,
 '05123361210000': 176954.915661254,
 '